1、Transforms

对传入的图片进行大小、像素等的变换，使之满足神经网络的输入要求。

target_transform

对标签类型进行变换，后处理。

*以上都需要在DataSet中去定义好，通过get_item去得到变换后的、正确的、符合模型规范的特征和标签。

2、torch.nn提供了我们构建神经网络所需要的所有模块，都是nn.Module的子类。

3、如何去构建一个分类模型（基于手写字识别）

In [4]:
#1、导入一些基本的库
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

In [5]:
#2、确定在什么设备上训练模型
device="cuda" if torch.cuda.is_available() else "cpu" #判断cuda是否可用
print(f"Using {device} device")

Using cuda device


In [10]:
#3、定义分类模型网络   12分钟时
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten=nn.Flatten()
        self.liner_relu_stack=nn.Sequential(
            nn.Linear(28*28, 512),  #第0层，线性层/MLP/前馈神经网络，两个参数分别表示输入特征的特殊维度，输出的隐含层的大小
            nn.ReLU(),              #非线性激活函数
            nn.Linear(512, 512),    #第2层，##为什么要用这么多Liner和ReLU  分三层
            nn.ReLU(),
            nn.Linear(512, 10),     #第4层，logits，维度为10
        )
    def forward(self, x):   #代表模块的前向运算
        x=self.flatten(x)
        logits=self.flatten(x)
        return logits

In [11]:
#4、实例化model
model=NeuralNetwork().to(device)  
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (liner_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [12]:
#5、使用model
X=torch.rand(1, 28, 28, device=device)
logits=model(X)
pred_probab=nn.Softmax(dim=1)(logits) #预测的概率
y_pred=pred_probab.argmax(1) #对概率求出最大值，得到样本的分类值
print(f"Predicted class:{y_pred}")

Predicted class:tensor([34], device='cuda:0')


4、该模型的层

In [14]:
#1、看一下输入张量的大小
input_image=torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [15]:
#2、查看Flatten的效果
flatten=nn.Flatten() #从第一维到最后一维浓缩成一个维度
flat_image=flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [16]:
#3、nn.Linear
layer1=nn.Linear(in_features=28*28 ,out_features=20)
hidden1=layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [17]:
#4、非线性层nn.ReLU
print(f"Before ReLU: {hidden1}\n\n")
hidden1=nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3758,  0.2062, -0.1051,  0.3165,  0.3148,  0.0767,  0.2469,  0.5818,
         -0.4367, -0.3949, -0.3052,  0.0841, -0.0545, -0.0668,  0.2776,  0.9790,
          0.5659, -0.4810, -0.1284, -0.3587],
        [ 0.2905,  0.1356, -0.1982,  0.2233,  0.4671,  0.0014,  0.4075,  0.0942,
         -0.1422, -0.4951,  0.0769, -0.0526,  0.4538, -0.0665, -0.1765,  0.4787,
          0.3414, -0.2654, -0.1421,  0.1242],
        [ 0.3523,  0.2653, -0.4260,  0.1836,  0.3698,  0.2172,  0.1737,  0.2413,
         -0.3954, -0.2000,  0.4418,  0.0600,  0.0892, -0.0521,  0.1705,  1.0458,
          0.0155, -0.2811, -0.1302, -0.0715]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3758, 0.2062, 0.0000, 0.3165, 0.3148, 0.0767, 0.2469, 0.5818, 0.0000,
         0.0000, 0.0000, 0.0841, 0.0000, 0.0000, 0.2776, 0.9790, 0.5659, 0.0000,
         0.0000, 0.0000],
        [0.2905, 0.1356, 0.0000, 0.2233, 0.4671, 0.0014, 0.4075, 0.0942, 0.0000,
         0.0000, 0.0769, 0.0000, 0.4538, 0.0000, 0.00

In [18]:
#5、nn.Sequential  #模块有序的容器
seq_modules=nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image=torch.rand(3,28,28)
logits=seq_modules(input_image)

In [19]:
#6、nn.Softmax
softmax=nn.Softmax(dim=1)
pred_probab=softmax(logits) #关于10个类的分类后的概率分布

In [20]:
#7、打印模型的所有参数
print(f"Model structure: {model}\n\n")

for name,param in model.named_parameters(): #torch.nn.model父类的一个方法，返回一个字典，元组
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (liner_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: liner_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0219,  0.0222, -0.0201,  ..., -0.0052,  0.0288, -0.0349],
        [ 0.0306,  0.0259,  0.0101,  ..., -0.0106, -0.0114,  0.0074]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer: liner_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0325, -0.0255], device='cuda:0', grad_fn=<SliceBackward0>)

Layer: liner_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0430,  0.0202,  0.0166,  ...,  0.0061,  0.0030, -0.0060],
        [ 0.0317, -0.0050,  0.0259,  ..., -0.0275, -0.0270, -0.0076]],
       device='cuda:0', grad_fn=<SliceBackwa